## Week 3
### Importing dependencies

In [ ]:
import torch
import torchvision
from torchvision import transforms

from torch import nn
import torch.nn.functional as F
from torch.optim import AdamW

### Preparing the data

In [2]:
# A set of transformations to apply to the images
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize(0.485, 0.229)
])

# Load the training dataset (the data is now "readied"; it is not actually loaded into memory)
train_data = torchvision.datasets.FER2013(root='./fer2013', train=True, download=True, transform=transforms)

# Create a data loader for (actually) loading the data in batches
train_loader = torch.utils.data.DataLoader(train_data, batch_size=19, shuffle=True, num_workers=2)

True

## Week 4
### Define the model architecture

In [ ]:
class EmotionModel(nn.Module):
    def __init__(self):
        super(EmotionModel, self).__init__()
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),     # padding=1 keeps the image size the same before and after convolution
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),          # [batch_size, 32, 48, 48] => [batch_size, 32, 24, 24]
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),          # [batch_size, 64, 24, 24] => [batch_size, 64, 12, 12]
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)           # [batch_size, 128, 12, 12] => [batch_size, 128, 6, 6]
        )
        
        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 6 * 6, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 7)
        )
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)   # [batch_size, flattened_size]
        x = self.fc_layers(x)
        return x
            